In [1]:
# Set constants
DATASET = "O3 Run"
ALLOW_ONLY_HP_MATCHES = True
TRIALS_REQUIRED = 3
DIFFERENCE_THRESHS = [1.05, 1.1, 1.5, 3, 4.5]

In [2]:
# Load data
import pandas as pd

data = pd.read_csv("latent_trials.csv")
data.head()

/tmp/ipykernel_133052/3332622169.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Classifier,Based on AE,sigma,precision,recall,f-score,max_iter,auc,n_estimators,max_depth,...,gamma,kernel,probability,algorithm,leaf_size,n_neighbors,contamination,n_bins,sigma_base,sigma_mult
0,ReconstructionThreshold,20240209142141,0.8,0.440816,0.720000,0.546835,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ReconstructionThreshold,20240209142141,1.0,0.403509,0.306667,0.348485,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ReconstructionThreshold,20240209142141,1.2,0.090909,0.006667,0.012422,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ReconstructionThreshold,20240209142141,1.4,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ReconstructionThreshold,20240209142141,1.6,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Add processed classifer column
data["Processed Classifier"] = [classifier + "_%.2f" % sigma if classifier == "ReconstructionThreshold" else classifier for classifier, sigma in data[["Classifier", "sigma"]].values]
# data["Processed Classifier"] = data["Classifier"]
data = data.drop(columns=["Classifier"])

data["Dataset"] = [DATASET for _ in data.index]

In [4]:
# Add AE data

ae_data = pd.read_csv("ae_trials.csv", keep_default_na=False)
ae_data = ae_data[["name", "type", "input_size", "attempt", "leader" , "masking", "mask_size"]]
ae_data["Primary Model"] = ["%d" % float(leader) if leader else "%d" % float(base) for base, leader in ae_data[["name", "leader"]].values]
ae_data = ae_data.drop(columns=["leader"])

ae_data["type"] = [type + " (%.2f Masked)" % mask_size if mask else type for type, mask, mask_size in ae_data[["type", "masking", "mask_size"]].values]

ae_data.head()

,name,type,input_size,attempt,masking,mask_size,Primary Model
0,20240209142141,Adversarial AE (0.50 Masked),4096,0,True,0.50,20240209142141
1,20240209160358,Vanilla AE (0.50 Masked),4096,0,True,0.50,20240209142141
2,20240212171643,Adversarial AE (0.25 Masked),4096,0,True,0.25,20240212171643
3,20240212193050,Vanilla AE (0.25 Masked),4096,0,True,0.25,20240212171643
4,20240212195117,Adversarial AE (0.25 Masked),4096,1,True,0.25,20240212171643


In [5]:
# Convert AE data to dict and add to ll data

ae_data = {name: [name, ae_type, input_size, attempt, primary, masking, mask_size] for name, ae_type, input_size, attempt, masking, mask_size, primary in ae_data.values}
data = data[[name in ae_data.keys() for name in data["Based on AE"].values]]
data[["AE", "Type", "Input Size", "Attempt", "Primary AE Model", "Mask", "Mask Size"]] = [ae_data[based_on] for based_on in data["Based on AE"].values.tolist()]

data[data["Processed Classifier"] == "Discriminator_Thresh_30"]["Type"]
data.head()

,Based on AE,sigma,precision,recall,f-score,max_iter,auc,n_estimators,max_depth,min_samples_leaf,...,sigma_mult,Processed Classifier,Dataset,AE,Type,Input Size,Attempt,Primary AE Model,Mask,Mask Size
0,20240209142141,0.8,0.440816,0.720000,0.546835,NaN,NaN,NaN,NaN,NaN,...,NaN,ReconstructionThreshold_0.80,O3 Run,20240209142141,Adversarial AE (0.50 Masked),4096,0,20240209142141,True,0.5
1,20240209142141,1.0,0.403509,0.306667,0.348485,NaN,NaN,NaN,NaN,NaN,...,NaN,ReconstructionThreshold_1.00,O3 Run,20240209142141,Adversarial AE (0.50 Masked),4096,0,20240209142141,True,0.5
2,20240209142141,1.2,0.090909,0.006667,0.012422,NaN,NaN,NaN,NaN,NaN,...,NaN,ReconstructionThreshold_1.20,O3 Run,20240209142141,Adversarial AE (0.50 Masked),4096,0,20240209142141,True,0.5
3,20240209142141,1.4,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,ReconstructionThreshold_1.40,O3 Run,20240209142141,Adversarial AE (0.50 Masked),4096,0,20240209142141,True,0.5
4,20240209142141,1.6,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,ReconstructionThreshold_1.60,O3 Run,20240209142141,Adversarial AE (0.50 Masked),4096,0,20240209142141,True,0.5


In [6]:
# Create groups and check if hyper-params are the same
hyper_params_to_check = [
    "max_iter",
    "n_estimators",
    "max_depth",
    "min_samples_leaf",
    "numEstimators",
    "C",
    "gamma",
    "kernel",
    "probability",
    "algorithm",
    "leaf_size",
    "n_neighbors",
    "n_bins",
    # "sigma_base",
    # "sigma_mult",
    "sigma",
]

data_uniqueness = data[["Primary AE Model", "Processed Classifier"] + hyper_params_to_check]
data_uniqueness = data_uniqueness.groupby(["Primary AE Model", "Processed Classifier"], group_keys=False).nunique()
data_uniqueness["HP Match"] = [all([n<=1 for n in v]) for v in data_uniqueness.values]

data_uniqueness = {(i[0], i[1]): same_hp for i, same_hp in zip(data_uniqueness.index.values, data_uniqueness["HP Match"].values)}

data["HP Match"] = [data_uniqueness[b_ae, pc] for b_ae, pc in data[["Primary AE Model", "Processed Classifier"]].values]
data["Classifier"] = data["Processed Classifier"]
data = data.drop(columns=["Processed Classifier"])

data = data[["Dataset", "Type", "Input Size", "Primary AE Model", "AE", "Classifier", "Attempt", "HP Match", "precision", "recall", "f-score", "auc"]]

data.head()

,Dataset,Type,Input Size,Primary AE Model,AE,Classifier,Attempt,HP Match,precision,recall,f-score,auc
0,O3 Run,Adversarial AE (0.50 Masked),4096,20240209142141,20240209142141,ReconstructionThreshold_0.80,0,True,0.440816,0.720000,0.546835,NaN
1,O3 Run,Adversarial AE (0.50 Masked),4096,20240209142141,20240209142141,ReconstructionThreshold_1.00,0,True,0.403509,0.306667,0.348485,NaN
2,O3 Run,Adversarial AE (0.50 Masked),4096,20240209142141,20240209142141,ReconstructionThreshold_1.20,0,True,0.090909,0.006667,0.012422,NaN
3,O3 Run,Adversarial AE (0.50 Masked),4096,20240209142141,20240209142141,ReconstructionThreshold_1.40,0,True,0.000000,0.000000,0.000000,NaN
4,O3 Run,Adversarial AE (0.50 Masked),4096,20240209142141,20240209142141,ReconstructionThreshold_1.60,0,True,0.000000,0.000000,0.000000,NaN


In [7]:
# Rename p, r and f1
data = data.rename(columns={"precision": "Precision", "recall": "Recall", "f-score": "F1-Score", "auc": "Area Under Curve"})

In [8]:
# Create final table
def row_max(df):
    max_f1 = df["F1-Score"].max()
    return df[df["F1-Score"] == max_f1].iloc[0]

if ALLOW_ONLY_HP_MATCHES:
    data = data[data["HP Match"]]

data = data[["Dataset", "Type", "Input Size", "Primary AE Model", "AE", "Classifier", "Precision", "Recall", "F1-Score", "Area Under Curve"]]
data_std = data.groupby(["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier"])[["Precision", "Recall", "F1-Score", "Area Under Curve"]].std()
aes = data.groupby(["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier"])["AE"].apply(lambda x : ":".join(["%d" % i for i in x]))
data = data.groupby(["Dataset", "Type", "Input Size", "Primary AE Model", "Classifier"])[["Precision", "Recall", "F1-Score", "Area Under Curve"]].mean()

data[["Precision Standard Deviation", "Recall Standard Deviation", "F1-Score Standard Deviation", "Area Under Curve Standard Deviation"]] = data_std[["Precision", "Recall", "F1-Score", "Area Under Curve"]]
data["Autoencoders"] = aes
data["AE Count"] = [len(v.split(":")) for v in data["Autoencoders"].values]
print(data["AE Count"].max())
data = data[data["AE Count"] >= TRIALS_REQUIRED]

data = data[~((data["Recall"] == 1) & (data["Precision"] < 0.6))]

data = data.groupby(["Dataset", "Input Size", "Type", "Classifier"]).apply(row_max)

formated_strings = ["%.4f (%.4f, %.4f)" % (f, p, r) for p, r, f in data[["Precision", "Recall", "F1-Score"]].values]
data["Formated Results"] = formated_strings
data = data.sort_values(["Type", "Input Size", "F1-Score"], ascending=False)

data.head()

4


Precision  \
Dataset Input Size Type                     Classifier                                
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80   0.462409   
                                            XGBClassifier                  0.572205   
                                            LogisticRegression             0.560245   
                                            GradientBoostingClassifier     0.562272   
                                            ExtraTreesClassifier           0.506312   

                                                                            Recall  \
Dataset Input Size Type                     Classifier                               
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80  0.773690   
                                            XGBClassifier                 0.605690   
                                            LogisticRegression            0.598471   
                                            GradientBoostingClassifier    0.554554   
                                            ExtraTreesClassifier          0.595223   

                                                                          F1-Score  \
Dataset Input Size Type                     Classifier                               
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80  0.578683   
                                            XGBClassifier                 0.576591   
                                            LogisticRegression            0.569841   
                                            GradientBoostingClassifier    0.545197   
                                            ExtraTreesClassifier          0.541804   

                                                                          Area Under Curve  \
Dataset Input Size Type                     Classifier                                       
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80               NaN   
                                            XGBClassifier                         0.592611   
                                            LogisticRegression                    0.557470   
                                            GradientBoostingClassifier            0.548749   
                                            ExtraTreesClassifier                  0.576354   

                                                                          Precision Standard Deviation  \
Dataset Input Size Type                     Classifier                                                   
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80                      0.021750   
                                            XGBClassifier                                     0.076560   
                                            LogisticRegression                                0.046177   
                                            GradientBoostingClassifier                        0.054779   
                                            ExtraTreesClassifier                              0.082999   

                                                                          Recall Standard Deviation  \
Dataset Input Size Type                     Classifier                                                
O3 Run  4096       Vanilla AE (0.75 Masked) ReconstructionThreshold_0.80                   0.067585   
                                            XGBClassifier                                  0.130436   
                                            LogisticRegression                             0.126555   
                                            GradientBoostingClassifier                     0.184193   
                                            ExtraTreesClassifier                           0.162994   

                                                                          F1-Score Standard Deviation  \
Dataset Input Size Type      

In [9]:
# Save final table

data.to_csv("Final LL Classifiers Table.csv")